In [1]:
import geopandas

### ex_1 prep
http://pp-import-production.s3.amazonaws.com/WDPA_Manual_1_5.pdf

In [ ]:
wpa_gdf = geopandas.read_file("wdpa_aug2019-shapefile/WDPA_Aug2019-shapefile-polygons.shp")
wpa_gdf.head()

In [ ]:
len(wpa_gdf)

In [ ]:
sa_gdf = geopandas.read_file("data/SouthAmerica.shp")
sa_gdf.head()

In [ ]:
sapa_gdf = wpa_gdf[wpa_gdf.ISO3.isin(set(sa_gdf.iso_alpha3))]
sapa_gdf.head()

In [ ]:
len(sapa_gdf)

In [ ]:
#sapa_gdf.to_file("SAPA_Aug2019-shapefile-polygons.shp")

### ex_2 prep (webscraping)

In [11]:
import requests
import lxml.html as lh
import pandas as pd

In [12]:
url='https://stats-japan.com/t/kiji/13392'

# Create a handle, page, to handle the contents of the website
page = requests.get(url)

# Store the contents of the website under doc
doc = lh.fromstring(page.content)

# Parse data that are stored between <tr>..</tr> of HTML
t_elements = doc.xpath('//tr')

In [13]:
# t_elements[6] contains first row
col=[]
i=0
for t in t_elements[6]:
    i+=1
    name=t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"1"
2:"Tokyo"
3:"12,868,000人"
4:"88.83"


In [14]:
# t_elements[52] contains final row
col=[]
i=0
for t in t_elements[52]:
    i+=1
    name=t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"47"
2:"Tottori"
3:"591,000人"
4:"41.89"


In [38]:
# t_elements[4] contains titles
col=[]
i=-1
for t in t_elements[4]:
    name=t.text_content()
    if i >= 0:
        print('%d:"%s"'%(i,name))
        col.append((name,[]))
    i+=1

for j in range(6,53):
    T=t_elements[j]
    
    # i is the index of our column
    i=-1
    
    # Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        if i==0:
            col[i][1].append(data)
        if i==1:
            col[i][1].append(int(data[:-1].replace(",","")))
        if i==2:
            col[i][1].append(float(data))
        #Increment i for the next column
        i+=1

0:"Prefectures"
1:"Population"
2:"StandardScore"


In [39]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [40]:
df.head()

,Prefectures,Population,StandardScore
0,Tokyo,12868000,88.83
1,Kanagawa,8943000,73.82
2,Osaka,8801000,73.28
3,Aichi,7418000,67.99
4,Saitama,7130000,66.89


In [41]:
df.dtypes

Prefectures       object
Population         int64
StandardScore    float64
dtype: object

In [42]:
df.to_csv('japan-population-per-prefecture.csv')

### ex_3 prep / prefecture file

In [2]:
japan_gdf = geopandas.read_file("PopDensity/Pop_density.shp")
japan_gdf.head()

,ID_1,NAME_1,ENGTYPE_1,PrefPopDen,geometry
0,1457,Aichi,Prefecture,1405.0,(POLYGON ((137.0952290000001 34.65329700000012...
1,1458,Akita,Prefecture,99.0,(POLYGON ((139.5572519999999 39.20329800000007...
2,1459,Aomori,Prefecture,150.0,"(POLYGON ((141.398605 40.92472200000003, 141.3..."
3,1460,Chiba,Prefecture,1175.0,(POLYGON ((139.8248750000001 34.98967300000004...
4,1461,Ehime,Prefecture,259.0,"(POLYGON ((132.5585940000001 32.9122430000001,..."


In [4]:
japan_gdf = japan_gdf[["NAME_1", "geometry"]]

In [5]:
japan_gdf.head()

,NAME_1,geometry
0,Aichi,(POLYGON ((137.0952290000001 34.65329700000012...
1,Akita,(POLYGON ((139.5572519999999 39.20329800000007...
2,Aomori,"(POLYGON ((141.398605 40.92472200000003, 141.3..."
3,Chiba,(POLYGON ((139.8248750000001 34.98967300000004...
4,Ehime,"(POLYGON ((132.5585940000001 32.9122430000001,..."


In [6]:
japan_gdf.rename(columns={'NAME_1':'prefecture'}, 
                 inplace=True)

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [8]:
#japan_gdf.to_file('japan-prefecture-boundaries.shp')

In [10]:
len(japan_gdf)

47